# [NOME DO PROJETO] - Análise Exploratória de Dados

**Objetivo:** Descreva aqui o objetivo principal desta análise.

**Fonte dos Dados:** Indique a origem dos dados (ex: Sistema de Vendas, Relatório de Marketing, etc.).

**Data:** 17 de agosto de 2025

## 1. Configuração do Ambiente

Nesta seção, importamos as bibliotecas e configuramos o ambiente para uma melhor visualização dos dados.

In [ ]:
# Bibliotecas de manipulação de dados
import pandas as pd
import numpy as np

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Kit do Analista
from ferramentas_analista import carregar_dados, converter_notebook_para_py

# --- Configurações de Visualização ---
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None) # Descomente se precisar ver todas as linhas
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Ambiente configurado com sucesso!")

## 2. Carga de Dados

Utilizamos a função `carregar_dados` para importar os dados de forma inteligente (CSV, Excel, JSON, etc.).

In [ ]:
caminho_arquivo_dados = 'caminho/para/seu/arquivo.csv'
df = carregar_dados(caminho_arquivo_dados)

if df is not None:
    df_analise = df.copy()

## 3. Inspeção Inicial dos Dados (Dossiê do Dataset)

Esta é a etapa mais crucial para 'sentir' os dados. Realizamos uma série de verificações básicas para criar um dossiê completo sobre o dataset que recebemos.

In [ ]:
if 'df_analise' in locals() and df_analise is not None:
    print(f"O dataset contém {df_analise.shape[0]} linhas e {df_analise.shape[1]} colunas.\n")
    
    print("--- 1. Visualização das Primeiras e Últimas Linhas ---")
    display(df_analise.head())
    display(df_analise.tail())

    print("\n--- 2. Nomes de Todas as Colunas ---")
    print(df_analise.columns.tolist())
    
    print("\n--- 3. Tipos de Dados e Uso de Memória ---")
    df_analise.info(memory_usage='deep')
    
    print("\n--- 4. Análise de Linhas Duplicadas (em colunas seguras) ---")
    colunas_problematicas = []
    for col in df_analise.columns:
        if df_analise[col].apply(lambda x: isinstance(x, (list, dict))).any():
            colunas_problematicas.append(col)
    
    if colunas_problematicas:
        print(f"Aviso: As seguintes colunas contêm listas/dicionários e serão ignoradas na checagem: {colunas_problematicas}")
    
    colunas_para_verificar = [col for col in df_analise.columns if col not in colunas_problematicas]
    num_duplicatas = df_analise.duplicated(subset=colunas_para_verificar).sum()
    
    if num_duplicatas > 0:
        print(f"\033[91mAVISO: Foram encontradas {num_duplicatas} linhas duplicadas.\033[0m")
    else:
        print("✅ Nenhuma linha duplicada encontrada.")
        
    print("\n--- 5. Análise de Valores Únicos (Cardinalidade) ---")
    print("Contagem de valores únicos por coluna (ajuda a identificar IDs e constantes):")
    display(df_analise[colunas_para_verificar].nunique().sort_values(ascending=True).to_frame(name='Contagem de Únicos'))
    
    print("\n--- 6. Análise de Valores Nulos (Percentual) ---")
    percent_nulos = (df_analise.isnull().sum() / len(df_analise) * 100).sort_values(ascending=False)
    percent_nulos = percent_nulos[percent_nulos > 0]
    if not percent_nulos.empty:
        print("Colunas com valores nulos:")
        display(percent_nulos.to_frame(name='Percentual de Nulos (%)'))
    else:
        print("✅ Nenhuma coluna com valores nulos encontrada.")
        
    print("\n--- 7. Estatísticas Descritivas (Colunas Numéricas) ---")
    display(df_analise.describe())
    
    print("\n--- 8. Estatísticas Descritivas (Colunas Categóricas) ---")
    display(df_analise.describe(include=['object', 'category']))

## 4. Limpeza e Pré-Processamento (Checklist de ETL)

Com base no dossiê acima, tratamos os problemas identificados. Descomente e adapte os blocos de código que forem relevantes para o seu projeto.

In [ ]:
# --- 4.1 Seleção e Remoção de Colunas ---
# # Opção A: Remover colunas irrelevantes pelo nome
# colunas_para_remover = ['id_transacao', 'coluna_inutil']
# df_analise.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

# # Opção B: Selecionar apenas as colunas que você quer manter
# colunas_para_manter = ['id_cliente', 'data_venda', 'produto', 'preco']
# df_analise = df_analise[colunas_para_manter]

# # Opção C (Avançado): Usar .filter() para selecionar colunas por padrão no nome
# df_ids = df_analise.filter(like='id') # Pega todas as colunas que contêm 'id' no nome

# --- 4.2 Renomeação de Colunas ---
# # Renomear colunas específicas para nomes mais claros e sem acentos/espaços
# df_analise.rename(columns={'NM_CLIENTE': 'nome_cliente', 'DT_VENDA': 'data_venda'}, inplace=True)

# # Padronizar todos os nomes de colunas (ex: para minúsculas e snake_case)
# df_analise.columns = df_analise.columns.str.strip().str.lower().str.replace(' ', '_', regex=False).str.replace('(', '', regex=False).str.replace(')', '', regex=False)

# --- 4.3 Tratamento de Dados Ausentes ---
# # Estratégia 1: Remover linhas onde colunas CRÍTICAS são nulas
# df_analise.dropna(subset=['id_cliente', 'valor_compra'], inplace=True)

# # Estratégia 2: Preencher nulos com um valor específico (medidas de tendência central ou constantes)
# df_analise['idade'].fillna(df_analise['idade'].median(), inplace=True)
# df_analise['categoria_produto'].fillna('Desconhecida', inplace=True)

# --- 4.4 Correção de Tipos de Dados ---
# # Converter colunas de data que foram lidas como texto
# df_analise['data_venda'] = pd.to_datetime(df_analise['data_venda'], errors='coerce') # 'coerce' transforma datas inválidas em NaT (Not a Time)

# # Exemplo prático: Converter uma coluna de preço (ex: 'R$ 1.234,56') para número
# if 'preco' in df_analise.columns and df_analise['preco'].dtype == 'object':
#     df_analise['preco'] = df_analise['preco'].str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

# # Converter para tipos de dados que economizam memória
# df_analise['status'] = df_analise['status'].astype('category')

# --- 4.5 Tratamento de Duplicatas ---
# # Remover linhas que são inteiramente duplicadas (usando o subset seguro definido na inspeção)
# df_analise.drop_duplicates(subset=colunas_para_verificar, inplace=True)

# --- 4.6 Criação de Novas Colunas (Engenharia de Features) ---
# # Extrair informações de datas
# if 'data_venda' in df_analise.columns:
#     df_analise['ano_venda'] = df_analise['data_venda'].dt.year
#     df_analise['mes_venda'] = df_analise['data_venda'].dt.month

# # Criar categorias a partir de uma variável numérica (Binning)
# df_analise['faixa_etaria'] = pd.cut(df_analise['idade'], bins=[0, 18, 35, 60, 100], labels=['Jovem', 'Adulto', 'Meia-Idade', 'Idoso'])

# --- 4.7 Limpeza e Padronização de Strings ---
# # Remover espaços em branco no início e no fim de uma coluna de texto
# df_analise['nome_produto'] = df_analise['nome_produto'].str.strip()

# # Padronizar para letras minúsculas
# df_analise['cidade'] = df_analise['cidade'].str.lower()

print("Etapa de limpeza e pré-processamento concluída. Verifique os resultados abaixo.")
# df_analise.info() # É uma boa prática verificar os tipos de dados novamente após a limpeza

## 5. Transformação e Agrupamento de Dados

Após a limpeza, podemos começar a transformar os dados, criando agregações e juntando informações de diferentes fontes para responder perguntas de negócio.

In [ ]:
# --- 5.1 Agrupamentos com .groupby() ---
# # Objetivo: Calcular estatísticas agregadas para diferentes categorias.
# # Exemplo: Calcular o preço médio, mínimo e máximo por categoria de produto.
# df_agrupado_categoria = df_analise.groupby('categoria_produto').agg(
#     preco_medio=('preco', 'mean'),
#     preco_maximo=('preco', 'max'),
#     contagem=('produto', 'count')
# ).sort_values(by='preco_medio', ascending=False)
# 
# display(df_agrupado_categoria)

# --- 5.2 Tabelas Dinâmicas com .pivot_table() ---
# # Objetivo: Reorganizar os dados em formato de matriz, similar a uma tabela dinâmica do Excel.
# # Exemplo: Ver o faturamento total por `ano` e `mes`.
# df_pivot = pd.pivot_table(
#     df_analise,
#     values='faturamento',
#     index='ano_venda',
#     columns='mes_venda',
#     aggfunc='sum',
#     fill_value=0 # Preenche meses sem vendas com 0
# )
# 
# display(df_pivot)

# --- 5.3 Combinação de Datasets com .merge() ---
# # Objetivo: Enriquecer o dataset principal com informações de outras tabelas.
# # Exemplo: Juntar os dados de vendas (df_analise) com um DataFrame de informações de clientes (df_clientes).

# # Criando um DataFrame de exemplo para clientes
# dados_clientes = {'id_cliente': [101, 102, 103], 'regiao_cliente': ['Sudeste', 'Nordeste', 'Sudeste']}
# df_clientes = pd.DataFrame(dados_clientes)

# # Juntando os dois DataFrames pela coluna em comum ('id_cliente')
# df_completo = pd.merge(
#     df_analise, 
#     df_clientes, 
#     on='id_cliente', # Chave para a junção
#     how='left'      # Mantém todos os registros de vendas, mesmo que não encontre cliente correspondente
# )
# 
# display(df_completo.head())

## 6. Exportação dos Dados Tratados

Após a limpeza e transformação, é uma excelente prática salvar o DataFrame resultante. Isso cria um "checkpoint" de dados limpos que pode ser usado em outras análises, relatórios ou modelos, sem precisar repetir todo o processo de ETL.

In [ ]:
# --- Salvando em CSV (.to_csv) ---
# # SUGESTÃO: Use ';' como separador e encoding 'utf-8-sig' para máxima compatibilidade com Excel.
# caminho_saida_csv = 'dados_analise_limpos.csv'
# df_analise.to_csv(
#     caminho_saida_csv,
#     index=False,          # Para não salvar o índice do DataFrame como uma coluna no arquivo
#     sep=';',              # Ponto e vírgula é bem reconhecido pelo Excel no Brasil
#     decimal=',',          # Define a vírgula como separador decimal
#     encoding='utf-8-sig'  # 'sig' (Byte Order Mark) ajuda o Excel a reconhecer a codificação UTF-8
# )
# print(f"DataFrame limpo salvo em '{caminho_saida_csv}'")

# --- Salvando em Excel (.to_excel) ---
# # Útil para compartilhar com áreas de negócio que não usam Python.
# caminho_saida_excel = 'dados_analise_limpos.xlsx'
# df_analise.to_excel(
#     caminho_saida_excel,
#     index=False,
#     sheet_name='Dados Principais'
# )
# print(f"DataFrame limpo salvo em '{caminho_saida_excel}'")

## 7. Análise Exploratória de Dados (EDA)

Com os dados limpos, investigamos e visualizamos os dados para encontrar padrões e insights.

### 7.1 Análise Univariada
Análise de cada variável individualmente para entender sua distribuição e características.

In [ ]:
# --- Para variáveis NUMÉRICAS --- 
# SUGESTÃO: Use Histogramas para ver a distribuição e Boxplots para identificar outliers.

# Exemplo com Plotly (interativo)
# coluna_numerica = 'sua_coluna_numerica'
# fig = px.histogram(df_analise, x=coluna_numerica, title=f'Distribuição de {coluna_numerica}', nbins=50, marginal='box')
# fig.show()


In [ ]:
# --- Para variáveis CATEGÓRICAS ---
# SUGESTÃO: Use Gráficos de Barras para ver a frequência de cada categoria.

# Exemplo com Plotly (interativo)
# coluna_categorica = 'sua_coluna_categorica'
# contagem = df_analise[coluna_categorica].value_counts().reset_index()
# contagem.columns = [coluna_categorica, 'contagem']
# fig = px.bar(contagem, y=coluna_categorica, x='contagem', title=f'Contagem por {coluna_categorica}', orientation='h', text='contagem')
# fig.update_layout(yaxis={'categoryorder':'total ascending'})
# fig.show()

### 7.2 Análise Bivariada
Análise da relação entre pares de variáveis.

In [ ]:
# --- Relação entre duas variáveis NUMÉRICAS ---
# SUGESTÃO: Use Gráfico de Dispersão (Scatter Plot) para identificar correlações.

# Exemplo com Plotly (interativo)
# fig = px.scatter(df_analise, x='coluna_numerica_X', y='coluna_numerica_Y', 
#                  title='Relação entre X e Y', 
#                  hover_data=['outra_coluna_para_tooltip'], # Adiciona mais infos ao passar o mouse
#                  trendline='ols') # Adiciona uma linha de tendência
# fig.show()

In [ ]:
# --- Relação entre uma variável NUMÉRICA e uma CATEGÓRICA ---
# SUGESTÃO: Use Boxplots ou Violin Plots para comparar a distribuição da variável numérica entre as categorias.

# Exemplo com Plotly (interativo)
# fig = px.box(df_analise, x='coluna_categorica', y='coluna_numerica', 
#              title='Distribuição Numérica por Categoria', 
#              points='all') # Mostra todos os pontos de dados
# fig.show()

### 7.3 Análise Multivariada
Análise da relação entre três ou mais variáveis.

In [ ]:
# SUGESTÃO: Use Gráfico de Dispersão com atributos de cor ou tamanho para adicionar uma terceira dimensão.

# Exemplo com Plotly (interativo)
# fig = px.scatter(df_analise, x='coluna_numerica_X', y='coluna_numerica_Y', 
#                  color='coluna_categorica',  # Terceira dimensão (categórica) por cor
#                  size='outra_coluna_numerica', # Quarta dimensão (numérica) por tamanho da bolha
#                  title='Análise Multivariada')
# fig.show()

## 8. Conclusões e Próximos Passos

In [ ]:
print("Escreva aqui os principais insights e próximos passos da análise.")

## 9. Exportação do Script

Convertemos este notebook em um script Python limpo.

In [ ]:
# Certifique-se de que o nome do arquivo corresponde ao seu notebook
converter_notebook_para_py('notebook_modelo.ipynb')